In [1]:
import re
import nltk
import isodate
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MinMaxScaler

In [2]:
scaler = MinMaxScaler()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df = pd.read_csv(r"C:\Users\Lenovo\Downloads\mydata.csv", on_bad_lines='warn', engine='python')
df

,date,clock,stadium,class,attendance,Home Team,Goals Home,Away Team,Away Goals,home_possessions,...,away_duels,home_saves,away_saves,home_fouls,away_fouls,home_yellow,away_yellow,home_red,away_red,links
0,28th May 2023,4:30pm,Emirates Stadium,h,"60,095",2,5,13,0,51.0,...,52.2,0,3,8,11,0,0,0,0,https://www.skysports.com/football/arsenal-vs-...
1,28th May 2023,4:30pm,Villa Park,h,"42,212",7,2,6,1,40.3,...,47.8,3,3,15,16,4,4,0,0,https://www.skysports.com/football/aston-villa...
2,28th May 2023,4:30pm,Gtech Community Stadium,h,"17,120",9,1,1,0,34.4,...,50.0,2,3,12,8,4,0,0,0,https://www.skysports.com/football/brentford-v...
3,28th May 2023,4:30pm,Stamford Bridge,d,"40,130",12,1,4,1,64.4,...,45.5,3,5,9,11,0,0,0,0,https://www.skysports.com/football/chelsea-vs-...
4,28th May 2023,4:30pm,Selhurst Park,d,"25,198",11,1,16,1,66.0,...,41.7,3,2,9,13,0,2,0,0,https://www.skysports.com/football/crystal-pal...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,13/09/2020,2:00pm,The Hawthorns,a,0,24,0,18,3,35.8,...,36.0,4,1,12,9,1,1,0,0,https://www.skysports.com/football/west-bromwi...
1136,12/9/2020,8:00pm,London Stadium,a,0,14,0,4,2,58.3,...,59.5,0,3,13,7,2,2,0,0,https://www.skysports.com/football/west-ham-un...
1137,12/9/2020,5:30pm,Anfield,h,0,5,4,19,3,48.8,...,46.2,0,3,9,6,1,0,0,0,https://www.skysports.com/football/liverpool-v...
1138,12/9/2020,3:00pm,Selhurst Park,h,0,11,1,20,0,29.4,...,50.0,5,2,14,11,2,1,0,0,https://www.skysports.com/football/crystal-pal...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              1140 non-null   object 
 1   clock             1140 non-null   object 
 2   stadium           1140 non-null   object 
 3   class             1140 non-null   object 
 4   attendance        1140 non-null   object 
 5   Home Team         1140 non-null   int64  
 6   Goals Home        1140 non-null   int64  
 7   Away Team         1140 non-null   int64  
 8   Away Goals        1140 non-null   int64  
 9   home_possessions  1140 non-null   float64
 10  away_possessions  1140 non-null   float64
 11  home_shots        1140 non-null   int64  
 12  away_shots        1140 non-null   int64  
 13  home_on           1140 non-null   int64  
 14  away_on           1140 non-null   int64  
 15  home_off          1140 non-null   int64  
 16  away_off          1140 non-null   int64  


In [6]:
import re

def remove_date_suffix(date_str):
    # Remove st, nd, rd, th from dates
    if isinstance(date_str, str):
        return re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
    return date_str

df['clean_date'] = df['date'].apply(remove_date_suffix)


In [7]:
df['parsed_date'] = pd.to_datetime(df['clean_date'], dayfirst=True, errors='coerce')


In [8]:
print(df[df['parsed_date'].isna()][['date']])


                date
380    22nd May 2022
381    22nd May 2022
382    22nd May 2022
383    22nd May 2022
384    22nd May 2022
...              ...
1135      13/09/2020
1136       12/9/2020
1137       12/9/2020
1138       12/9/2020
1139       12/9/2020

[390 rows x 1 columns]


In [ ]:
# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Filter for matches played in 2021
df_2021 = df[df['date'].dt.year == 2021]

In [ ]:
# Create a result column
df_2021['result'] = df_2021.apply(lambda row: 
    'home_win' if row['Goals Home'] > row['Away Goals'] 
    else 'away_win' if row['Goals Home'] < row['Away Goals'] 
    else 'draw', axis=1)

# Count each result type
result_counts = df_2021['result'].value_counts()
print(result_counts)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=df_2021, x='result', order=['home_win', 'draw', 'away_win'], palette='pastel')
plt.title("Match Outcomes in Premier League 2021")
plt.xlabel("Match Result")
plt.ylabel("Number of Matches")
plt.show()


In [ ]:
print("Avg goals by home team:", df_2021['Goals Home'].mean())
print("Avg goals by away team:", df_2021['Away Goals'].mean())


In [ ]:
result_percent = result_counts / result_counts.sum() * 100
print(result_percent.round(2))


In [ ]:
# Extract year column
df['year'] = df['date'].dt.year

# Group by year and count matches
season_match_counts = df.groupby('year').size()
print(season_match_counts)


In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [ ]:
df['year'] = df['date'].dt.year.astype('Int64')  # Allows NaNs as well


In [ ]:
season_match_counts = df['year'].value_counts().sort_index()
print(season_match_counts)


In [ ]:
df